In [1]:
import json
import types
from collections import OrderedDict
from pathlib import Path
from pprint import pprint

import torch
import torch.nn as nn

from dataset import get_new_dataloader
from funcs import get_dataloaders
from models import PreActResNet18, WideResNet
from train import epoch, epoch_test


In [10]:
# constant
CWD_PATH = Path.cwd()
MODEL_DIR_PATH = CWD_PATH.parent / "models/CIFAR10"
LABELS = [
    "S1A",
    "I1A",
    "D1A",
    "S2A",
    "I2A",
    "D2A",
    "S3A",
    "I3A",
    "D3A",
    "S4A",
    "I4A",
    "D4A",
    "S1B",
    "I1B",
    "D1B",
    "S2B",
    "I2B",
    "D2B",
    "S3B",
    "I3B",
    "D3B",
    "S4B",
    "I4B",
    "D4B",
]

L1__MODEL_TYPE_MAP = {
    "S": "teacher",
    "I": "independent",
    "D": "pre-act-18",
}

L1__MODEL_TYPE_DISPLAY_MAP = {
    "S": "Teacher",
    "I": "Independent",
    "D": "Diff. Arch.",
}

L2__MODEL_NORM_MAP = {
    "1": True,
    "2": True,
    "3": False,
    "4": False,
}

L2__DATA_NORM_MAP = {
    "1": True,
    "2": False,
    "3": True,
    "4": False,
}

L3__LOAD_NORM_MAP = {
    "A": True,
    "B": False,
}

In [3]:
def load_model(
    model_type: str,
    location: str,
    model_normalize: bool,
):
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")

    # load model CIFAR10
    if model_type == "pre-act-18":
        model = PreActResNet18(num_classes=10, normalize=model_normalize)
    elif model_type == "independent":
        model = WideResNet(
            n_classes=10,
            depth=16,  # deep_full for CIFAR10
            widen_factor=1,
            normalize=model_normalize,
            dropRate=0.3,
        )
    else:
        model = WideResNet(
            n_classes=10,
            depth=28,  # deep_full for CIFAR10
            widen_factor=10,
            normalize=model_normalize,
            dropRate=0.3,
        )

    try:
        model = model.to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    except:
        model = nn.DataParallel(model).to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    model.eval()
    return model


In [4]:
model_name__acc = OrderedDict()

In [5]:
# model performance in training
for label in LABELS:
    l1 = label[0]
    l2 = label[1]
    l3 = label[2]

    model_type_str = L1__MODEL_TYPE_MAP[l1]
    model_norm_str = "normalized" if L2__MODEL_NORM_MAP[l2] else "unnormalized"
    data_norm_str = "normalized" if L2__DATA_NORM_MAP[l2] else "unnormalized"
    load_norm_str = "normalized" if L3__LOAD_NORM_MAP[l3] else "unnormalized"

    if not L3__LOAD_NORM_MAP[l3]:
        continue

    model_name = f"model_{model_type_str}_model-{model_norm_str}_data-{data_norm_str}"
    model_path = MODEL_DIR_PATH / model_name / "final.pt"
    print(model_name)
    model = load_model(
        model_type=model_type_str,
        location=str(model_path),
        model_normalize=L2__MODEL_NORM_MAP[l2],
    )

    train_loader, test_loader = get_dataloaders(
        dataset="CIFAR10",
        batch_size=1000,
        pseudo_labels=False,
        normalize=L2__DATA_NORM_MAP[l2],
        concat=False,
        concat_factor=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }


model_teacher_model-normalized_data-normalized
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:42<00:00,  1.17it/s]


0.99998
0.945
model_independent_model-normalized_data-normalized
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:15<00:00,  3.13it/s]


0.58676
1.0
model_pre-act-18_model-normalized_data-normalized
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


...

0.99998
0.9394
model_independent_model-unnormalized_data-unnormalized
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:12<00:00,  3.89it/s]


0.60064
1.0
model_pre-act-18_model-unnormalized_data-unnormalized
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:15<00:00,  3.32it/s]


0.99922
0.9328


In [9]:
pprint(model_name__acc)

# Serializing json
json_object = json.dumps(model_name__acc, indent=2)
 
# Writing to sample.json
with open("training_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('S1A', {'test_acc': 0.945, 'train_acc': 0.99998}),
             ('I1A', {'test_acc': 1.0, 'train_acc': 0.58676}),
             ('D1A', {'test_acc': 0.9302, 'train_acc': 0.99882}),
             ('S2A', {'test_acc': 0.9423, 'train_acc': 1.0}),
             ('I2A', {'test_acc': 1.0, 'train_acc': 0.59254}),
             ('D2A', {'test_acc': 0.9321, 'train_acc': 0.99938}),
             ('S3A', {'test_acc': 0.9431, 'train_acc': 0.99998}),
             ('I3A', {'test_acc': 1.0, 'train_acc': 0.60016}),
             ('D3A', {'test_acc': 0.9331, 'train_acc': 0.99912}),
             ('S4A', {'test_acc': 0.9394, 'train_acc': 0.99998}),
             ('I4A', {'test_acc': 1.0, 'train_acc': 0.60064}),
             ('D4A', {'test_acc': 0.9328, 'train_acc': 0.99922})])


In [4]:
model_name__acc = OrderedDict()

# model performance in FE
for label in LABELS:
    l1 = label[0]
    l2 = label[1]
    l3 = label[2]

    model_type_str = L1__MODEL_TYPE_MAP[l1]
    model_norm_str = "normalized" if L2__MODEL_NORM_MAP[l2] else "unnormalized"
    data_norm_str = "normalized" if L2__DATA_NORM_MAP[l2] else "unnormalized"
    load_norm_str = "normalized" if L3__LOAD_NORM_MAP[l3] else "unnormalized"

    model_name = f"model_{model_type_str}_model-{model_norm_str}_data-{data_norm_str}"
    model_path = MODEL_DIR_PATH / model_name / "final.pt"
    print(model_name)
    model = load_model(
        model_type=model_type_str,
        location=str(model_path),
        model_normalize=L3__LOAD_NORM_MAP[l3],
    )

    train_loader, test_loader = get_dataloaders(
        dataset="CIFAR10",
        batch_size=1000,
        pseudo_labels=False,
        normalize=False,
        train_shuffle=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }



model_teacher_model-normalized_data-normalized
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:32<00:00,  1.52it/s]


0.40282
0.3979
model_independent_model-normalized_data-normalized
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:06<00:00,  7.41it/s]


0.13956
0.1402
model_pre-act-18_model-normalized_data-normalized
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


...

0.99998
0.9394
model_independent_model-unnormalized_data-unnormalized
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:06<00:00,  7.55it/s]


0.74124
1.0
model_pre-act-18_model-unnormalized_data-unnormalized
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:08<00:00,  5.58it/s]


0.9995
0.9328


In [5]:
pprint(model_name__acc)

# Serializing json
json_object = json.dumps(model_name__acc, indent=2)
 
# Writing to sample.json
with open("fe_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('S1A', {'test_acc': 0.3979, 'train_acc': 0.40282}),
             ('I1A', {'test_acc': 0.1402, 'train_acc': 0.13956}),
             ('D1A', {'test_acc': 0.3942, 'train_acc': 0.39834}),
             ('S2A', {'test_acc': 0.9423, 'train_acc': 1.0}),
             ('I2A', {'test_acc': 1.0, 'train_acc': 0.74484}),
             ('D2A', {'test_acc': 0.9321, 'train_acc': 0.99948}),
             ('S3A', {'test_acc': 0.9414, 'train_acc': 1.0}),
             ('I3A', {'test_acc': 0.9974, 'train_acc': 0.7242}),
             ('D3A', {'test_acc': 0.9305, 'train_acc': 0.99816}),
             ('S4A', {'test_acc': 0.3921, 'train_acc': 0.4027}),
             ('I4A', {'test_acc': 0.3634, 'train_acc': 0.31868}),
             ('D4A', {'test_acc': 0.2142, 'train_acc': 0.2161}),
             ('S1B', {'test_acc': 0.1003, 'train_acc': 0.09894}),
             ('I1B', {'test_acc': 0.1, 'train_acc': 0.1}),
             ('D1B', {'test_acc': 0.1, 'train_acc': 0.1}),
             ('S2B', {'test_acc': 0.51

## Read

In [9]:
with open("fe_performance.json", "r") as outfile:
    data = json.load(outfile)
    for model_label, acc_dict in data.items():
        print(f"{acc_dict['test_acc']:.3f}")

0.398
0.140
0.394
0.942
1.000
0.932
0.941
0.997
0.930
0.392
0.363
0.214
0.100
0.100
0.100
0.519
0.182
0.596
0.377
0.146
0.453
0.939
1.000
0.933


In [11]:
for label in LABELS:
    l1 = label[0]
    l2 = label[1]
    l3 = label[2]

    model_type_str = L1__MODEL_TYPE_MAP[l1]
    model_norm_str = "normalized" if L2__MODEL_NORM_MAP[l2] else "unnormalized"
    data_norm_str = "normalized" if L2__DATA_NORM_MAP[l2] else "unnormalized"
    load_norm_str = "normalized" if L3__LOAD_NORM_MAP[l3] else "unnormalized"

    file_name = f"model_{model_type_str}_model-{model_norm_str}_data-{data_norm_str}_load-model-{load_norm_str}"
    key_name = f"{model_type_str}_model-{model_norm_str}_data-{data_norm_str}_load-model-{load_norm_str}"
    display_name = f"{L1__MODEL_TYPE_DISPLAY_MAP[l1]} MODEL-{model_norm_str} DATA-{data_norm_str} LOAD-model-{load_norm_str}"
    print(display_name)
    # print(key_name)

Teacher MODEL-normalized DATA-normalized LOAD-model-normalized
Independent MODEL-normalized DATA-normalized LOAD-model-normalized
Diff. Arch. MODEL-normalized DATA-normalized LOAD-model-normalized
Teacher MODEL-normalized DATA-unnormalized LOAD-model-normalized
Independent MODEL-normalized DATA-unnormalized LOAD-model-normalized
Diff. Arch. MODEL-normalized DATA-unnormalized LOAD-model-normalized
Teacher MODEL-unnormalized DATA-normalized LOAD-model-normalized
Independent MODEL-unnormalized DATA-normalized LOAD-model-normalized
Diff. Arch. MODEL-unnormalized DATA-normalized LOAD-model-normalized
Teacher MODEL-unnormalized DATA-unnormalized LOAD-model-normalized
Independent MODEL-unnormalized DATA-unnormalized LOAD-model-normalized
Diff. Arch. MODEL-unnormalized DATA-unnormalized LOAD-model-normalized
Teacher MODEL-normalized DATA-normalized LOAD-model-unnormalized
Independent MODEL-normalized DATA-normalized LOAD-model-unnormalized
Diff. Arch. MODEL-normalized DATA-normalized LOAD-mode